In [ ]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
from src.data import *
from src.space import *
from src.mcmc import *

level = 'cntyvtd'
proposal = 'planc2100'
# D = Data()
S = Space(level=level, proposal=proposal)
M = MCMC(gpickle=S.gpickle, nodes=S.tbls['nodes'],
         max_steps   = 0,
         random_seed = 0,
         defect_cap  = 1000000,
         pop_deviation_target=200)
M.run_chain()

In [11]:
try:
    import pandas_bokeh
except:
    os.system('pip install --upgrade pandas-bokeh')
    import pandas_bokeh
pandas_bokeh.output_notebook()

plan_cols = ['geoid', 'plan', 'district']
query = f"select {', '.join(plan_cols)} from {M.output}"
df = run_query(query).pivot(index='geoid', columns='plan', values='district')#.astype(int)
repl = {c:str(c) for c in df.columns}
df.rename(columns=repl, inplace=True)

geo_cols = ['geoid', 'polygon', 'county', 'total_pop', 'aland', 'density', 'polsby_popper']
query = f"select {', '.join(geo_cols)} from {M.output} where plan=0"
shapes = run_query(query).set_index('geoid')
geo = gpd.GeoSeries.from_wkt(shapes['polygon'], crs=crs_census).simplify(0.01).buffer(0) #<-- little white space @ .001 ~5.7 mb, minimal at .0001 ~10mb, with no white space ~37mb
gdf = shapes.drop(columns='polygon').join(df, how='inner')
gdf = gpd.GeoDataFrame(gdf, geometry=geo)

Loading BokehJS ...

BadRequest: 400 Unrecognized name: county; Did you mean point? at [1:24]

(job ID: 0519eb84-b0f8-4b16-93a4-029a9c0b27fe)

                                                     -----Query Job SQL Follows-----                                                      

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:select geoid, polygon, county, total_pop, aland, density, polsby_popper from cmat-315920.TX_cd_planc2100.cntyvtd_0_0_all where plan=0
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |

In [10]:
fig = gdf.plot_bokeh(
    figsize = (900, 600),
    slider = list(repl.values()),
    slider_name = "PLAN #",
    show_colorbar = False,
    colorbar_tick_format="0",
    colormap = "Category20",
    # hovertool_string = '@geoid, @county<br>pop=@total_pop<br>density=@density{0.0}<br>land=@aland{0.0}<br>pp=@polsby_popper{0.0}',
    tile_provider = "CARTODBPOSITRON",
    return_html = True,
    show_figure = True,
    **{'fill_alpha' :.5,
      'line_alpha':.05,}
)
fig

'<script src="https://cdn.bokeh.org/bokeh/release/bokeh-2.4.0.min.js"></script>\n<script src="https://cdn.bokeh.org/bokeh/release/bokeh-gl-2.4.0.min.js"></script>\n<script src="https://cdn.bokeh.org/bokeh/release/bokeh-widgets-2.4.0.min.js"></script>\n<script src="https://cdn.bokeh.org/bokeh/release/bokeh-tables-2.4.0.min.js"></script>\n<script src="https://cdn.bokeh.org/bokeh/release/bokeh-mathjax-2.4.0.min.js"></script>\n\n\n\n<div class="bk-root" id="f8406af3-5a74-400a-a723-6ba2d6b424ee" data-root-id="1265"></div>\n\n\n<script type="text/javascript">\n    (function() {\n          const fn = function() {\n            Bokeh.safely(function() {\n              (function(root) {\n                function embed_document(root) {\n                  \n                const docs_json = \'{"22446958-3e9c-4a92-afaa-4e371b703ba4":{"defs":[],"roots":{"references":[{"attributes":{},"id":"1200","type":"LinearScale"},{"attributes":{"coordinates":null,"data_source":{"id":"1241"},"glyph":{"id":"1243"}

In [5]:
gdf.plot_bokeh(category=0)

ValueError: failed to validate ColumnDataSource(id='1191', ...).data: expected an element of ColumnData(String, Seq(Any)), got {0: array([37, 38, 21, ..., 19, 30, 27]), 'Colormap': array([37, 38, 21, ..., 19, 30, 27]), 'index': array(['453000105', '201000431', '029003118', ..., '305000009',
       '113003602', '057000021'], dtype=object), '__x__': array([list([[[-10862028, -10864715, -10863822, -10862615, -10865815, -10862129, -10855518, -10851121, -10852753, -10856608, -10859789, -10862704, -10860496, -10862028]]]),
       list([[[-10630038, -10629315, -10627448, -10630248, -10630038]]]),
       list([[[-10957505, -10956966, -10956889, -10956376, -10957505]]]),
       ...,
       list([[[-11363038, -11338848, -11338862, -11327541, -11331880, -11333515, -11362415, -11363038]]]),
       list([[[-10784054, -10783976, -10785887, -10785427, -10784054]]]),
       list([[[-10764664, -10759819, -10758429, -10759810, -10757760, -10760494, -10764664]]])],
      dtype=object), '__y__': array([li

In [6]:
# cols = ['geoid', 'plan', 'polygon']
        
# # cols = ['geoid', 'county', 'total_pop', 'density', 'aland', 'perim', 'polsby_popper', 'polygon']
# query = f"select {', '.join(cols)} from {M.output} limit 10"
# df = run_query(query)

# df = df.pivot(index='geoid', columns='plan')#.astype(int)
# df
# geo = gpd.GeoSeries.from_wkt(df['polygon'], crs=crs_census)#.simplify(0.001).buffer(0) #<-- little white space @ .001 ~5.7 mb, minimal at .0001 ~10mb, with no white space ~37mb
# gdf = gpd.GeoDataFrame(df.drop(columns='polygon'), geometry=geo)
# fig = gdf.plot_bokeh(
#     figsize = (900, 600),
#     slider = plans,
#     slider_name = "PLAN #",
#     show_colorbar = False,
#     colorbar_tick_format="0",
#     colormap = "Category20",
#     hovertool_string = '@geoid, @county<br>pop=@total_pop<br>density=@density{0.0}<br>land=@aland{0.0}<br>pp=@polsby_popper{0.0}',
#     tile_provider = "CARTODBPOSITRON",
#     return_html = True,
#     show_figure = show,
#     **{'fill_alpha' :.5,
#       'line_alpha':.05,}
# )

In [ ]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
from src.data import *
from src.space import *
from src.mcmc import *

# D = Data()
#             refresh_all=('crosswalks', 'assignments', 'shapes', 'census', 'elections', 'all')
#             refresh_all=('census', 'all')
#             refresh_tbl=('all'))
#             refresh_tbl=('crosswalks')
#            )

S = Space(level='cntyvtd', proposal='planc2100',
#               refresh_all=('graph'))
#               refresh_all=('nodes', 'graph'),)
             )
# # print(self.proposal, self.district_type)
M = MCMC(gpickle=S.gpickle, nodes=S.tbls['nodes'], max_steps=0, random_seed=0, defect_cap=1000000, pop_deviation_target=200)
M.run_chain()
M.post_process()
# for district_type, proposals in D.proposals_dict.items():
#     for proposal in proposals:
#         print(f'\n\n{district_type} {proposal}')
#         self = Space(level='cntyvtd',
# #                     district_type=district_type,
#                     proposal=proposal,
# #                     refresh_tbl = ('nodes', 'graphs')
#                    )
#         assert 1==2